## Advanced Box Score Calculations
Author: Akshay Easwaran <akeaswaran@me.com>

---
Based on work by ESPN's Bill Connelly, Football Outsiders, and Football Study Hall

Data from CollegeFootballData.com

Useful Resources for Stat Definitions:
- Football Study Hall website: https://www.footballstudyhall.com/
- Football Study Hall book: https://www.amazon.com/Study-Hall-College-Football-Stories/dp/1484989961
- Football Outsiders: https://www.footballoutsiders.com/info/glossary
- Example Advanced Box Score: https://twitter.com/ESPN_BillC/status/1176572006969597954/photo/1

**How to use this Notebook**

1. Dump a game's play by play data and drive data from CollegeFootballData.com.
2. Change the file names in the next cell to match where your data files are located.
3. Change the away and home team names appropriately.
4. Hit Cell > Run All Cells.

In [46]:
import pandas as pd

import requests
import pandas as pd
import json
import html
import os.path

selected_team = 'Auburn'
selected_opponent = 'Alabama'
selected_week = 14
selected_team_is_home = True

def retrieveCfbData(endpoint, team, week):
    file_path = f"data/{endpoint if (endpoint != 'plays') else 'pbp'}/{endpoint[:-1] if (endpoint != 'plays') else 'pbp'}-data-{team.lower().replace(' ','-')}-wk{week}.json"
    if (os.path.exists(file_path)):
        return file_path
    res = requests.get(f"https://api.collegefootballdata.com/{endpoint}?seasonType=regular&year=2019&team={html.escape(team)}&week={week}")
    content = res.json()
    with open(file_path, 'w') as f:
        json.dump(content, f)
    return json.dumps(content)
    

def verify_division(num1, num2):
    if num2 == 0:
        return 0
    else:
        return num1 / num2

pbp_data = pd.read_json(retrieveCfbData('plays',selected_team, selected_week))
# pbp_data.info()

drives = pd.read_json(retrieveCfbData('drives',selected_team, selected_week))
drives.drop(['offense_conference','start_time','end_time','defense_conference','elapsed','game_id'], axis = 1, inplace=True) 
drives = drives[
    (~drives.drive_result.isin(['Uncategorized']))
#     & (~drives.drive_result.str.contains('END OF'))
]

away_team = selected_opponent if selected_team_is_home else selected_team
away_score = 45
home_team = selected_team if selected_team_is_home else selected_opponent
home_score = 48

In [47]:
# Data Cleaning
# Fix the bad yard line markers for away teams
drives.loc[
    drives.offense == away_team, ['start_yardline']
] = 100 - drives.start_yardline
drives.loc[
    drives.offense == away_team, ['end_yardline']
] = 100 - drives.end_yardline

print("Total Drives:", len(drives))

pbp_data = pbp_data[
    (pbp_data.down != 0)
]

# Drop some unnecessary columns
pbp_data.drop(['offense_conference','defense_conference','clock'], axis = 1, inplace=True) 
# Ignore some types of plays cause they're special teams and weird
ignore_types = ["Defensive 2pt Conversion","Blocked Field Goal","Blocked Punt","Missed Field Goal Return","Blocked Punt Touchdown","Missed Field Goal Return Touchdown","Extra Point Missed","Extra Point Good","Timeout","End of Half","End of Game","Uncategorized","Penalty","Kickoff","Kickoff Return (Offense)","Kickoff Return Touchdown","Punt", "Field Goal Good","Field Goal Missed","Safety"]
pbp_data = pbp_data[~(pbp_data.play_type.isin(ignore_types))]
base_pbp_data = pbp_data.copy()

# Eliminate garbage time plays
garbage_time_data = pbp_data[
    (pbp_data.down == 0)
    | ((abs(pbp_data.offense_score - pbp_data.defense_score) >= 38) & (pbp_data.period == 2))
    | ((abs(pbp_data.offense_score - pbp_data.defense_score) >= 28) & (pbp_data.period == 3))
    | ((abs(pbp_data.offense_score - pbp_data.defense_score) >= 22) & (pbp_data.period == 4))
]

print("Total Plays:", len(base_pbp_data))
print("Garbage Time Plays:", len(garbage_time_data))
print("% of plays in garbage time:", len(garbage_time_data)/len(pbp_data))
pbp_data.drop(garbage_time_data.index, inplace=True)
print("Non-Garbage Time Plays:", len(pbp_data))

# print("Total clean plays:", len(pbp_data))

Total Drives: 25
Total Plays: 142
Garbage Time Plays: 0
% of plays in garbage time: 0.0
Non-Garbage Time Plays: 142


In [48]:
import numpy as np

pbp_data.distance = pbp_data.distance.astype(float)

bad_types = ["Interception","Pass Interception Return","Interception Return Touchdown",'Fumble Recovery (Opponent)','Sack','Fumble Return Touchdown']

pbp_data.loc[
    ((pbp_data.play_type.isin(bad_types))
     & (~pbp_data.play_type.str.contains('Sack'))) ,['yards_gained']] = 0

# pbp_data.loc[
#     (pbp_data.play_type.isin(bad_types)) ,['yards_gained']] = 0

def is_successful(down, distance, yards_gained, play_type):
    if (play_type in bad_types):
        return False 
    if ((down == 1) & (yards_gained >= (0.5 * distance))):
        return True
    elif ((down == 2)) & (yards_gained >= (0.7 * distance)):
        return True
    elif ((down == 3) & (yards_gained >= distance)):
        return True
    elif ((down == 4) & (yards_gained >= distance)):
        return True
    else:
        return False
    
pbp_data['play_successful'] = np.vectorize(is_successful)(pbp_data.down, pbp_data.distance, pbp_data.yards_gained, pbp_data.play_type)
pbp_data.play_successful.value_counts()

base_pbp_data['play_successful'] = np.vectorize(is_successful)(base_pbp_data.down, base_pbp_data.distance, base_pbp_data.yards_gained, base_pbp_data.play_type)
base_pbp_data.play_successful.value_counts()

def is_explosive(yards_gained):
    if (yards_gained >= 15):
        return True
    else:
        return False
    
pbp_data['play_explosive'] = np.vectorize(is_explosive)(pbp_data.yards_gained)
base_pbp_data['play_explosive'] = np.vectorize(is_explosive)(base_pbp_data.yards_gained)

In [49]:
pass_types = ["Pass Reception","Pass Incompletion","Passing Touchdown","Interception","Pass Interception Return","Interception Return Touchdown","Sack"]
rush_types = ["Rush","Rushing Touchdown",'Fumble Recovery (Opponent)','Fumble Return Touchdown']

print("Success Rates")
print(away_team)
print("Overall:",len(base_pbp_data[(base_pbp_data.offense == away_team) & (base_pbp_data.play_successful == True)]) / len(base_pbp_data[(base_pbp_data.offense == away_team)]))
print("Passing:",len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_successful == True) & (pbp_data.play_type.isin(pass_types))]) / len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_type.isin(pass_types))]))
print("Rushing:",len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_successful == True) & (pbp_data.play_type.isin(rush_types))]) / len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_type.isin(rush_types))]))
print(home_team)
print("Overall:",len(base_pbp_data[(base_pbp_data.offense == home_team) & (base_pbp_data.play_successful == True)]) / len(base_pbp_data[(base_pbp_data.offense == home_team)]))
print("Passing:",len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_successful == True) & (pbp_data.play_type.isin(pass_types))]) / len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_type.isin(pass_types))]))
print("Rushing:",len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_successful == True) & (pbp_data.play_type.isin(rush_types))]) / len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_type.isin(rush_types))]))

print("")
print("Success Rate by Q")
for i in range(1, base_pbp_data.period.nunique()+1):
    print(f"Q{i}\n{away_team}:", len(base_pbp_data[(base_pbp_data.period == i) & (base_pbp_data.offense == away_team) & (base_pbp_data.play_successful == True)]) / len(base_pbp_data[(base_pbp_data.period == i) & (base_pbp_data.offense == away_team)]),f"\n{home_team}:", len(base_pbp_data[(base_pbp_data.period == i) & (base_pbp_data.offense == home_team) & (base_pbp_data.play_successful == True)]) / len(base_pbp_data[(base_pbp_data.period == i) & (base_pbp_data.offense == home_team)]))

print("")
print("Success Rate by Down")
for i in range(1, base_pbp_data.down.nunique()+1):
    print(f"Down {i}\n{away_team}:", verify_division(len(base_pbp_data[(base_pbp_data.down == i) & (base_pbp_data.offense == away_team) & (base_pbp_data.play_successful == True)]), len(base_pbp_data[(base_pbp_data.down == i) & (base_pbp_data.offense == away_team)])),f"\n{home_team}:", verify_division(len(base_pbp_data[(base_pbp_data.down == i) & (base_pbp_data.offense == home_team) & (base_pbp_data.play_successful == True)]), len(base_pbp_data[(base_pbp_data.down == i) & (base_pbp_data.offense == home_team)])))
    

Success Rates
Alabama
Overall: 0.4675324675324675
Passing: 0.425
Rushing: 0.5135135135135135
Auburn
Overall: 0.36923076923076925
Passing: 0.36666666666666664
Rushing: 0.35294117647058826

Success Rate by Q
Q1
Alabama: 0.5263157894736842 
Auburn: 0.45454545454545453
Q2
Alabama: 0.42105263157894735 
Auburn: 0.4166666666666667
Q3
Alabama: 0.4166666666666667 
Auburn: 0.2857142857142857
Q4
Alabama: 0.5333333333333333 
Auburn: 0.3125

Success Rate by Down
Down 1
Alabama: 0.43243243243243246 
Auburn: 0.46875
Down 2
Alabama: 0.5 
Auburn: 0.2
Down 3
Alabama: 0.42857142857142855 
Auburn: 0.38461538461538464
Down 4
Alabama: 1.0 
Auburn: 0


In [50]:
# Havoc plays
havoc_plays = base_pbp_data[
    (((base_pbp_data.play_type == 'Pass Incompletion')
    & (base_pbp_data.play_text.str.contains('broken up', regex=False)))
    | (base_pbp_data.play_type == 'Fumble Recovery (Opponent)')
    | (base_pbp_data.play_type == 'Sack')
    | (base_pbp_data.play_type.str.contains('Interception', regex=False))
    | (base_pbp_data.yards_gained < 0))
    & (base_pbp_data.play_type != 'Penalty')
]

print(away_team,"Havoc Rate: ", len(havoc_plays[
    havoc_plays.defense == away_team
]), "/", len(base_pbp_data[
    base_pbp_data.defense == away_team
]), "(",len(havoc_plays[
    havoc_plays.defense == away_team
]) / len(base_pbp_data[
    base_pbp_data.defense == away_team
]),")")

print(home_team,"Havoc Rate: ", len(havoc_plays[
    havoc_plays.defense == home_team
]), "/", len(base_pbp_data[
    base_pbp_data.defense == home_team
]), "(",len(havoc_plays[
    havoc_plays.defense == home_team
]) / len(base_pbp_data[
    base_pbp_data.defense == home_team
]),")")

Alabama Havoc Rate:  7 / 65 ( 0.1076923076923077 )
Auburn Havoc Rate:  11 / 77 ( 0.14285714285714285 )


In [51]:
away_team_drives = drives[
    drives.offense == away_team
]

print(away_team, "Drives:", len(away_team_drives))
print(away_team,"Yards:",sum(away_team_drives.yards))
print(away_team,"Plays:",sum(away_team_drives.plays))
print(away_team, "Avg Starting Field Position:", sum(away_team_drives.start_yardline) / len(away_team_drives))
print(away_team, "Yards per Play:", sum(away_team_drives.yards) / sum(away_team_drives.plays))
print(away_team, "Plays per Drive:", sum(away_team_drives.plays) / len(away_team_drives))
print(away_team, "Yards per Drive:", sum(away_team_drives.yards) / len(away_team_drives))
print(away_team, "Points per Drive:", away_score / len(away_team_drives))
#away_team_drives
print("")
home_team_drives = drives[
    drives.offense == home_team
]

print(home_team, "Drives:", len(home_team_drives))
print(home_team,"Yards:",sum(home_team_drives.yards))
print(home_team,"Plays:",sum(home_team_drives.plays))
print(home_team, "Avg Starting Field Position:", sum(home_team_drives.start_yardline) / len(home_team_drives))
print(home_team, "Yards per Play:", sum(home_team_drives.yards) / sum(home_team_drives.plays))
print(home_team, "Plays per Drive:", sum(home_team_drives.plays) / len(home_team_drives))
print(home_team, "Yards per Drive:", sum(home_team_drives.yards) / len(home_team_drives))
print(home_team, "Points per Drive:", home_score / len(home_team_drives))
#home_team_drives

Alabama Drives: 13
Alabama Yards: 515
Alabama Plays: 79
Alabama Avg Starting Field Position: 31.153846153846153
Alabama Yards per Play: 6.518987341772152
Alabama Plays per Drive: 6.076923076923077
Alabama Yards per Drive: 39.61538461538461
Alabama Points per Drive: 3.4615384615384617

Auburn Drives: 12
Auburn Yards: 385
Auburn Plays: 69
Auburn Avg Starting Field Position: 33.416666666666664
Auburn Yards per Play: 5.579710144927536
Auburn Plays per Drive: 5.75
Auburn Yards per Drive: 32.083333333333336
Auburn Points per Drive: 4.0


In [52]:
# Measuring success rate for a single player
# pbp_data[
#     (pbp_data.play_text.str.contains("Quentin Harris"))
#     & (pbp_data.play_type.isin(pass_types))
#     & (~pbp_data.play_type.str.contains("Sack"))
# ].play_successful.value_counts(normalize=True)

In [53]:
# Standard vs Passing Downs success rates
# Success rate on standard downs == leverage rate

standard_downs = pbp_data[
    (pbp_data.down == 1)
    | ((pbp_data.down == 2) & (pbp_data.distance <= 7))
    | ((pbp_data.down == 3) & (pbp_data.distance <= 4))
    | ((pbp_data.down == 4) & (pbp_data.distance <= 4)) 
]

passing_downs = pbp_data[
    ((pbp_data.down == 2) & (pbp_data.distance >= 8))
    | ((pbp_data.down == 3) & (pbp_data.distance >= 5))
    | ((pbp_data.down == 4) & (pbp_data.distance >= 5)) 
]

pass_plays = base_pbp_data[
    base_pbp_data.play_type.isin(pass_types)
]
rush_plays = base_pbp_data[
    base_pbp_data.play_type.isin(rush_types)
]

In [54]:
print(away_team,"Success Rate on Std Downs:",len(standard_downs[
    (standard_downs.offense == away_team)
    & (standard_downs.play_successful == True)
]) / len(standard_downs[
    (standard_downs.offense == away_team)
]))

print(away_team,"Success Rate on Passing Downs:",len(passing_downs[
    (passing_downs.offense == away_team)
    & (passing_downs.play_successful == True)
]) / len(passing_downs[
    (passing_downs.offense == away_team)
]))

print(home_team,"Success Rate on Std Downs:",len(standard_downs[
    (standard_downs.offense == home_team)
    & (standard_downs.play_successful == True)
]) / len(standard_downs[
    (standard_downs.offense == home_team)
]))

print(home_team,"Success Rate on Passing Downs:",len(passing_downs[
    (passing_downs.offense == home_team)
    & (passing_downs.play_successful == True)
]) / len(passing_downs[
    (passing_downs.offense == home_team)
]))

Alabama Success Rate on Std Downs: 0.509090909090909
Alabama Success Rate on Passing Downs: 0.36363636363636365
Auburn Success Rate on Std Downs: 0.4358974358974359
Auburn Success Rate on Passing Downs: 0.2692307692307692


In [55]:
# Stuff Rate
stuffed_plays = rush_plays[
    (rush_plays.yards_gained <= 0)
    & (rush_plays.play_type != 'Sack')
]

away_team_stuffs = stuffed_plays[
    stuffed_plays.defense == away_team
]
print(away_team,"Defensive Stuff Rate: ",len(away_team_stuffs),"/",len(rush_plays[rush_plays.defense == away_team]),"(",len(away_team_stuffs)/len(rush_plays[rush_plays.defense == away_team]),")")

home_team_stuffs = stuffed_plays[
    stuffed_plays.defense == home_team
]
print(home_team,"Defensive Stuff Rate: ",len(home_team_stuffs),"/",len(rush_plays[rush_plays.defense == home_team]),"(",len(home_team_stuffs)/len(rush_plays[rush_plays.defense == home_team]),")")

Alabama Defensive Stuff Rate:  7 / 34 ( 0.20588235294117646 )
Auburn Defensive Stuff Rate:  6 / 37 ( 0.16216216216216217 )


In [56]:
# Opportunity Rate
rush_opps = rush_plays[
    (rush_plays.yards_gained >= 4)
]
print(away_team,"Rush Opp Rate: ",len(rush_opps[rush_opps.offense == away_team]),"/",len(rush_plays[rush_plays.offense == away_team]),"(",len(rush_opps[rush_opps.offense == away_team])/len(rush_plays[rush_plays.offense == away_team]),")")

print(home_team,"Rush Opp Rate: ",len(rush_opps[rush_opps.offense == home_team]),"/",len(rush_plays[rush_plays.offense == home_team]),"(",len(rush_opps[rush_opps.offense == home_team])/len(rush_plays[rush_plays.offense == home_team]),")")

Alabama Rush Opp Rate:  20 / 37 ( 0.5405405405405406 )
Auburn Rush Opp Rate:  14 / 34 ( 0.4117647058823529 )


In [57]:
##### Line Yards -- not adjusted for down/distance/opponent/shotgun
def adjust_strength_for_ol(yards_gained):
    if (yards_gained < 0):
        return yards_gained * 1.25
    elif ((yards_gained >= 0) & (yards_gained <= 3)):
        return yards_gained * 1.0
    elif ((yards_gained >= 4) & (yards_gained <= 6)):
        return yards_gained * 0.5
    else:
        return 0

rush_plays['line_yards'] = rush_plays.apply(lambda x: adjust_strength_for_ol(x.yards_gained), axis=1)
rush_plays['highlight_yards'] = rush_plays.apply(lambda x: x.yards_gained - x.line_yards, axis=1)

rush_opps['line_yards'] = rush_opps.apply(lambda x: adjust_strength_for_ol(x.yards_gained), axis=1)
rush_opps['highlight_yards'] = rush_opps.apply(lambda x: x.yards_gained - x.line_yards, axis=1)

rush_plays['line_yards'] = rush_plays.apply(lambda x: adjust_strength_for_ol(x.yards_gained), axis=1)
rush_plays['highlight_yards'] = rush_plays.apply(lambda x: x.yards_gained - x.line_yards, axis=1)

rush_opps['line_yards'] = rush_opps.apply(lambda x: adjust_strength_for_ol(x.yards_gained), axis=1)
rush_opps['highlight_yards'] = rush_opps.apply(lambda x: x.yards_gained - x.line_yards, axis=1)


print(away_team,"Line Yards per Carry:",sum(rush_plays[rush_plays.offense == away_team].line_yards) / len(rush_plays[rush_plays.offense == away_team]))
print(away_team,"Highlight Yards per Carry:",sum(rush_plays[rush_plays.offense == away_team].highlight_yards) / len(rush_plays[rush_plays.offense == away_team]))
print(away_team,"Yards per Carry:",sum(rush_plays[rush_plays.offense == away_team].yards_gained) / len(rush_plays[rush_plays.offense == away_team]))
print(away_team,"Hlt Yards per Opp:",sum(rush_opps[rush_opps.offense == away_team].highlight_yards) / len(rush_opps[rush_opps.offense == away_team]))

print("")
print(home_team,"Line Yards per Carry:",sum(rush_plays[rush_plays.offense == home_team].line_yards) / len(rush_plays[rush_plays.offense == home_team]))
print(home_team,"Highlight Yards per Carry:",sum(rush_plays[rush_plays.offense == home_team].highlight_yards) / len(rush_plays[rush_plays.offense == home_team]))
print(home_team,"Yards per Carry:",sum(rush_plays[rush_plays.offense == home_team].yards_gained) / len(rush_plays[rush_plays.offense == home_team]))
print(home_team,"Hlt Yards per Opp:",sum(rush_opps[rush_opps.offense == home_team].highlight_yards) / len(rush_opps[rush_opps.offense == home_team]))

# rush_plays.head()

Alabama Line Yards per Carry: 0.8716216216216216
Alabama Highlight Yards per Carry: 4.20945945945946
Alabama Yards per Carry: 5.081081081081081
Alabama Hlt Yards per Opp: 7.675

Auburn Line Yards per Carry: 0.5073529411764706
Auburn Highlight Yards per Carry: 5.022058823529412
Auburn Yards per Carry: 5.529411764705882
Auburn Hlt Yards per Opp: 11.964285714285714


/Users/akeaswaran/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/Users/akeaswaran/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/Users/akeaswaran/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [58]:
def get_filtered_rush_stats(team, title, startQuarter, endQuarter):
    rush_set = rush_plays[((rush_plays.offense == team) & ((rush_plays.period >= startQuarter) & (rush_plays.period <= endQuarter)))]
    rush_opp_set = rush_set[(rush_set.offense == team)]
    print(f"{team} {title}")
    print("Line Yards per Carry:",sum(rush_set.line_yards) / len(rush_set))
    print("Highlight Yards per Carry:",sum(rush_set.highlight_yards) / len(rush_set))
    print("Yards per Carry:",sum(rush_set.yards_gained) / len(rush_set))
    print("Hlt Yards per Opp:",sum(rush_opp_set.highlight_yards) / len(rush_opp_set))
    print("")

get_filtered_rush_stats(away_team, "1st Half", 1, 2)
get_filtered_rush_stats(away_team, "2nd Half", 3, 4)
print("---\n")
get_filtered_rush_stats(home_team, "1st Half", 1, 2)
get_filtered_rush_stats(home_team, "2nd Half", 3, 4)

Alabama 1st Half
Line Yards per Carry: 0.9861111111111112
Highlight Yards per Carry: 5.180555555555555
Yards per Carry: 6.166666666666667
Hlt Yards per Opp: 5.180555555555555

Alabama 2nd Half
Line Yards per Carry: 0.7631578947368421
Highlight Yards per Carry: 3.289473684210526
Yards per Carry: 4.052631578947368
Hlt Yards per Opp: 3.289473684210526

---

Auburn 1st Half
Line Yards per Carry: 0.125
Highlight Yards per Carry: 5.232142857142857
Yards per Carry: 5.357142857142857
Hlt Yards per Opp: 5.232142857142857

Auburn 2nd Half
Line Yards per Carry: 0.775
Highlight Yards per Carry: 4.875
Yards per Carry: 5.65
Hlt Yards per Opp: 4.875



In [59]:
# Expected and Adjusted Turnover Margins

adj_turnover_plays = base_pbp_data[
    (base_pbp_data.play_type.str.contains('Interception', regex=False))
    | ((base_pbp_data.play_type == 'Pass Incompletion')
    & (base_pbp_data.play_text.str.contains('broken up', regex=False)))
    | (base_pbp_data.play_type.str.contains('Fumble', regex=False))
]

adj_fum = 0.49 * len(adj_turnover_plays[
    (adj_turnover_plays.play_type.str.contains('Fumble', regex=False))
])

# away_team Adj Turnovers
away_team_tos = adj_turnover_plays[
    (adj_turnover_plays.offense == away_team)
    | (adj_turnover_plays.defense == away_team)
]

away_team_ints_def = len(away_team_tos[
   (away_team_tos.play_type.str.contains('Interception', regex=False))
    & (away_team_tos.defense == away_team)
])

away_team_ints_off = len(away_team_tos[
   (away_team_tos.play_type.str.contains('Interception', regex=False))
    & (away_team_tos.offense == away_team)
])

away_team_pds = len(away_team_tos[
   (away_team_tos.play_type == 'Pass Incompletion')
    & (away_team_tos.play_text.str.contains('broken up', regex=False))
    & (away_team_tos.offense == away_team)
])

away_team_fum_rec = away_team_tos[((away_team_tos.play_type == 'Fumble Recovery (Opponent)') & (away_team_tos.defense == away_team))| ((away_team_tos.play_type == 'Fumble Recovery (Own)') & (away_team_tos.offense == away_team))]
away_team_fum_lost = away_team_tos[(away_team_tos.play_type == 'Fumble Recovery (Opponent)') & (away_team_tos.offense == away_team)]

In [60]:
# home_team Adj Turnovers (broken)
home_team_tos = adj_turnover_plays[
    (adj_turnover_plays.offense == home_team)
    | (adj_turnover_plays.defense == home_team)
]

home_team_ints_def = len(home_team_tos[
   (home_team_tos.play_type.str.contains('Interception', regex=False))
    & (home_team_tos.defense == home_team)
])

home_team_ints_off = len(home_team_tos[
   (home_team_tos.play_type.str.contains('Interception', regex=False))
    & (home_team_tos.offense == home_team)
])

home_team_pds = len(home_team_tos[
   (home_team_tos.play_type == 'Pass Incompletion')
    & (home_team_tos.play_text.str.contains('broken up', regex=False))
    & (home_team_tos.offense == home_team)
])

home_team_fum_rec = home_team_tos[((home_team_tos.play_type == 'Fumble Recovery (Opponent)') & (home_team_tos.defense == home_team)) | ((home_team_tos.play_type == 'Fumble Recovery (Own)') & (home_team_tos.offense == home_team))]
home_team_fum_lost = home_team_tos[(home_team_tos.play_type == 'Fumble Recovery (Opponent)') & (home_team_tos.offense == home_team)]

print(away_team)
print("Def INTs:",away_team_ints_def)
print("Off INTs:",away_team_ints_off)
print("Off PDs:",(away_team_pds + away_team_ints_off))
print("Exp INTs:",0.22 * (away_team_pds + away_team_ints_off))
print("Fum Recovered:",len(away_team_fum_rec))
print("Fum Lost:",len(away_team_fum_lost))
print("Exp Fum:",adj_fum)
away_to = away_team_ints_off + len(away_team_fum_lost)
print("Actual TO:",away_to)
print("Actual TO Margin:",-1 * away_team_ints_off - len(away_team_fum_lost) + away_team_ints_def + len(away_team_fum_rec))
away_team_exp_to = 0.22 * (away_team_pds + away_team_ints_off) + adj_fum
print("Exp TO:",away_team_exp_to)

print("")

print(home_team)
print("Def INTs:",home_team_ints_def)
print("Off INTs:",home_team_ints_off)
print("Off PDs:",(home_team_pds + home_team_ints_off))
print("Exp INTs:",0.22 * (home_team_pds + home_team_ints_off))
print("Fum Recovered:",len(home_team_fum_rec))
print("Fum Lost:",len(home_team_fum_lost))
print("Exp Fum:",adj_fum)
home_to = home_team_ints_off + len(home_team_fum_lost)
print("Actual TO:",home_to)
print("Actual TO Margin:",-1 * home_team_ints_off - len(home_team_fum_lost) + home_team_ints_def + len(home_team_fum_rec))
home_team_exp_to = 0.22 * (home_team_pds + home_team_ints_off) + adj_fum
print("Exp TO:",home_team_exp_to)

Alabama
Def INTs: 0
Off INTs: 2
Off PDs: 2
Exp INTs: 0.44
Fum Recovered: 1
Fum Lost: 0
Exp Fum: 0.98
Actual TO: 2
Actual TO Margin: -1
Exp TO: 1.42

Auburn
Def INTs: 2
Off INTs: 0
Off PDs: 0
Exp INTs: 0.0
Fum Recovered: 1
Fum Lost: 1
Exp Fum: 0.98
Actual TO: 1
Actual TO Margin: 2
Exp TO: 0.98


In [61]:
# home_team_tos

In [62]:
away_team_exp_to_margin = (home_team_exp_to - away_team_exp_to)
print("Exp TO Margin for",away_team,":", away_team_exp_to_margin)
home_team_exp_to_margin = (away_team_exp_to - home_team_exp_to)
print("Exp TO Margin for",home_team,":", home_team_exp_to_margin)

print("TO Luck (pts) for",away_team,":", ((home_to - away_to) - away_team_exp_to_margin) * 5.0)
print("TO Luck (pts) for",home_team,":", ((away_to - home_to) - home_team_exp_to_margin) * 5.0)

Exp TO Margin for Alabama : -0.43999999999999995
Exp TO Margin for Auburn : 0.43999999999999995
TO Luck (pts) for Alabama : -2.8000000000000003
TO Luck (pts) for Auburn : 2.8000000000000003


In [63]:
# Scoring Opportunities
# Definition: roughly, any time you get inside the opponent's 35, you should probably score

scoring_opps = drives[
    ((drives.start_yardline + drives.yards) >= 60)
]

# away team's scoring opps
print("Scoring Opportunities (IE: Drives inside Opponent's 40)")
print(away_team)
away_team_scoring_opps = scoring_opps[
    scoring_opps.offense == away_team
]
print("Total:",len(away_team_scoring_opps))
print("Scored:",len(away_team_scoring_opps[away_team_scoring_opps.scoring == True]))
print("Opp Efficiency:",len(away_team_scoring_opps[away_team_scoring_opps.scoring == True]) / len(away_team_scoring_opps))
print("Opps/Drive:",len(away_team_scoring_opps) / len(drives[drives.offense == away_team]))
print("Points/Opp:",(away_score / len(away_team_scoring_opps)))
#print("SR%:",0 if len(away_team_scoring_opps) == 0 else (len(away_team_scoring_opps[away_team_scoring_opps.play_successful == True]) / len(away_team_scoring_opps)))

print("")
# home team's scoring opps
print(home_team)
home_team_scoring_opps = scoring_opps[
    scoring_opps.offense == home_team
]
print("Total:",len(home_team_scoring_opps))
print("Scored:",len(home_team_scoring_opps[home_team_scoring_opps.scoring == True]))
print("Opp Efficiency:", 0 if len(home_team_scoring_opps) == 0 else (len(home_team_scoring_opps[home_team_scoring_opps.scoring == True]) / len(home_team_scoring_opps)))
print("Opps/Drive:",len(home_team_scoring_opps) / len(drives[drives.offense == home_team]))
print("Points/Opp:",0 if len(home_team_scoring_opps) == 0 else (home_score / len(home_team_scoring_opps)))
# print("SR%:",0 if len(home_team_scoring_opps) == 0 else (len(home_team_scoring_opps[home_team_scoring_opps.play_successful == True]) / len(home_team_scoring_opps)))

Scoring Opportunities (IE: Drives inside Opponent's 40)
Alabama
Total: 9
Scored: 8
Opp Efficiency: 0.8888888888888888
Opps/Drive: 0.6923076923076923
Points/Opp: 5.0

Auburn
Total: 7
Scored: 7
Opp Efficiency: 1.0
Opps/Drive: 0.5833333333333334
Points/Opp: 6.857142857142857


In [64]:
def calculate_success_in_scoring_opps(team, opps):
    opp_ids = opps.id.unique()
    success = 0
    total = 0
    for opp_id in opp_ids:
        opp_set = base_pbp_data[(base_pbp_data.drive_id == opp_id) & (base_pbp_data.offense == team)]
        opp_s_rate = verify_division(len(opp_set[opp_set.play_successful == True]), len(opp_set))
        print(f"{team} SR% in opp {opp_id}: {opp_s_rate}")
        success += len(opp_set[opp_set.play_successful == True])
        total += len(opp_set)
    s_rate = 0 if total == 0 else (success / total)
    print(f"{team} total SR% in scoring opps: {s_rate}")
    print("")

calculate_success_in_scoring_opps(away_team, away_team_scoring_opps)
calculate_success_in_scoring_opps(home_team, home_team_scoring_opps)

Alabama SR% in opp 4011108652: 0.5555555555555556
Alabama SR% in opp 4011108656: 0.5714285714285714
Alabama SR% in opp 4011108659: 0
Alabama SR% in opp 40111086511: 0.5
Alabama SR% in opp 40111086513: 0.5
Alabama SR% in opp 40111086517: 0.45454545454545453
Alabama SR% in opp 40111086518: 0.6
Alabama SR% in opp 40111086522: 0.8333333333333334
Alabama SR% in opp 40111086524: 0.45454545454545453
Alabama total SR% in scoring opps: 0.5483870967741935

Auburn SR% in opp 4011108655: 1.0
Auburn SR% in opp 4011108657: 0.3
Auburn SR% in opp 40111086512: 0.7142857142857143
Auburn SR% in opp 40111086514: 0.4
Auburn SR% in opp 40111086516: 0.3333333333333333
Auburn SR% in opp 40111086521: 0.25
Auburn SR% in opp 40111086523: 0.45454545454545453
Auburn total SR% in scoring opps: 0.45652173913043476



In [65]:
# Rushing Stats
rush_carries = rush_plays[
    (~rush_plays.play_type.isin(['Fumble Recovery (Opponent)','Fumble Return Touchdown']))
]

# look these up manually
away_team_rushers = ["A.J. Davis","Vincent Davis","V'Lique Carter","Kenny Pickett","Maurice Ffrench","Shocky Jacques-Louis","Aaron Mathews"]
home_team_rushers = ["Jordan Mason","James Graham","Jerry Howard Jr","Lucas Johnson"]

print("team level rushing stats")
print(away_team)
print(len(rush_carries[rush_carries.offense == away_team]),"car",
  sum(rush_carries[rush_carries.offense == away_team].yards_gained),"yards",
  len(rush_carries[(rush_carries.offense == away_team) & (rush_carries.play_type == "Rushing Touchdown")]),"TDs",
  len(rush_plays[(rush_plays.offense == away_team) & (rush_plays.play_type.str.contains("Fumble"))]),"Fum",
  "(",
  verify_division(sum(rush_carries[(rush_carries.offense == away_team)].yards_gained), len(rush_carries[(rush_carries.offense == away_team)])),"YPC",",",
  verify_division(sum(rush_carries[(rush_carries.offense == away_team)].line_yards), len(rush_carries[(rush_carries.offense == away_team)])),"line YPC",",",
  "SR%:",verify_division(len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_successful == True) & (pbp_data.play_type.isin(rush_types))]), len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_type.isin(rush_types))])),",",
  "Opp Rate:",verify_division(len(rush_opps[(rush_opps.offense == away_team)]), len(rush_carries[(rush_carries.offense == away_team)])),",",
  "Hlt/Opp:",verify_division(sum(rush_carries[(rush_carries.offense == away_team)].yards_gained), len(rush_opps[(rush_opps.offense == away_team)])),",",
  "Stuff%:",verify_division(len(stuffed_plays[stuffed_plays.offense == away_team]), len(pbp_data[pbp_data.offense == away_team])),
  ")")
print(home_team)
print(len(rush_carries[rush_carries.offense == home_team]),"car",
  sum(rush_carries[rush_carries.offense == home_team].yards_gained),"yards",
  len(rush_carries[(rush_carries.offense == home_team) & (rush_carries.play_type == "Rushing Touchdown")]),"TDs",
  len(rush_plays[(rush_plays.offense == home_team) & (rush_plays.play_type.str.contains("Fumble"))]),"Fum",
  "(",
  verify_division(sum(rush_carries[(rush_carries.offense == home_team)].yards_gained), len(rush_carries[(rush_carries.offense == home_team)])),"YPC",",",
  verify_division(sum(rush_carries[(rush_carries.offense == home_team)].line_yards), len(rush_carries[(rush_carries.offense == home_team)])),"line YPC",",",
  "SR%:",verify_division(len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_successful == True) & (pbp_data.play_type.isin(rush_types))]), len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_type.isin(rush_types))])),",",
  "Opp Rate:",verify_division(len(rush_opps[(rush_opps.offense == home_team)]), len(rush_carries[(rush_carries.offense == home_team)])),",",
  "Hlt/Opp:",verify_division(sum(rush_carries[(rush_carries.offense == home_team)].highlight_yards), len(rush_opps[(rush_opps.offense == home_team)])),",",
  "Stuff%:",verify_division(len(stuffed_plays[stuffed_plays.offense == home_team]), len(pbp_data[pbp_data.offense == home_team])),
  ")")
    
print("")
print(away_team,"Rushing Stats")
for qb in away_team_rushers:
    print(qb + ":", 
          len(rush_carries[rush_carries.play_text.str.contains(qb)]),"car",
          sum(rush_carries[rush_carries.play_text.str.contains(qb)].yards_gained),"yards",
          len(rush_carries[(rush_carries.play_text.str.contains(qb)) & (rush_carries.play_type == "Rushing Touchdown")]),"TDs",
          len(rush_plays[(rush_plays.play_text.str.contains(qb)) & (rush_plays.play_type.str.contains("Fumble"))]),"Fum",
          "(",
          verify_division(sum(rush_carries[rush_carries.play_text.str.contains(qb)].yards_gained), len(rush_carries[rush_carries.play_text.str.contains(qb)])),"YPC",",",
          verify_division(sum(rush_carries[rush_carries.play_text.str.contains(qb)].line_yards), len(rush_carries[rush_carries.play_text.str.contains(qb)])),"line YPC",",",
          "SR%:",verify_division(len(pbp_data[(pbp_data.play_text.str.contains(qb)) & (pbp_data.play_successful == True) & (pbp_data.play_type.isin(rush_types))]), len(pbp_data[(pbp_data.play_text.str.contains(qb)) & (pbp_data.play_type.isin(rush_types))])),",",
          "Opp Rate:",verify_division(len(rush_opps[rush_opps.play_text.str.contains(qb)]), len(rush_carries[rush_carries.play_text.str.contains(qb)])),",",
          "Hlt/Opp:",verify_division(sum(rush_opps[rush_opps.play_text.str.contains(qb)].highlight_yards), len(rush_opps[rush_opps.play_text.str.contains(qb)])),
          ")")

print("")

print(home_team,"Rushing Stats")
for qb in home_team_rushers:
    print(qb + ":", 
          len(rush_carries[rush_carries.play_text.str.contains(qb)]),"car",
          sum(rush_carries[rush_carries.play_text.str.contains(qb)].yards_gained),"yards",
          len(rush_carries[(rush_carries.play_text.str.contains(qb)) & (rush_carries.play_type == "Rushing Touchdown")]),"TDs",
          len(rush_plays[(rush_plays.play_text.str.contains(qb)) & (rush_plays.play_type.str.contains("Fumble"))]),"Fum",
          "(",
          verify_division(sum(rush_carries[rush_carries.play_text.str.contains(qb)].yards_gained), len(rush_carries[rush_carries.play_text.str.contains(qb)])),"YPC",",",
          verify_division(sum(rush_carries[rush_carries.play_text.str.contains(qb)].line_yards), len(rush_carries[rush_carries.play_text.str.contains(qb)])),"line YPC",",",
          "SR%:",verify_division(len(pbp_data[(pbp_data.play_text.str.contains(qb)) & (pbp_data.play_successful == True) & (pbp_data.play_type.isin(rush_types))]), len(pbp_data[(pbp_data.play_text.str.contains(qb)) & (pbp_data.play_type.isin(rush_types))])),",",
          "Opp Rate:",verify_division(len(rush_opps[rush_opps.play_text.str.contains(qb)]), len(rush_carries[rush_carries.play_text.str.contains(qb)])),",",
          "Hlt/Opp:",verify_division(sum(rush_opps[rush_opps.play_text.str.contains(qb)].highlight_yards), len(rush_opps[rush_opps.play_text.str.contains(qb)])),
          ")")


team level rushing stats
Alabama
37 car 188 yards 1 TDs 0 Fum ( 5.081081081081081 YPC , 0.8716216216216216 line YPC , SR%: 0.5135135135135135 , Opp Rate: 0.5405405405405406 , Hlt/Opp: 9.4 , Stuff%: 0.07792207792207792 )
Auburn
33 car 176 yards 2 TDs 1 Fum ( 5.333333333333333 YPC , 0.5227272727272727 line YPC , SR%: 0.35294117647058826 , Opp Rate: 0.42424242424242425 , Hlt/Opp: 11.339285714285714 , Stuff%: 0.1076923076923077 )

Alabama Rushing Stats
A.J. Davis: 0 car 0 yards 0 TDs 0 Fum ( 0 YPC , 0 line YPC , SR%: 0 , Opp Rate: 0 , Hlt/Opp: 0 )
Vincent Davis: 0 car 0 yards 0 TDs 0 Fum ( 0 YPC , 0 line YPC , SR%: 0 , Opp Rate: 0 , Hlt/Opp: 0 )
V'Lique Carter: 0 car 0 yards 0 TDs 0 Fum ( 0 YPC , 0 line YPC , SR%: 0 , Opp Rate: 0 , Hlt/Opp: 0 )
Kenny Pickett: 0 car 0 yards 0 TDs 0 Fum ( 0 YPC , 0 line YPC , SR%: 0 , Opp Rate: 0 , Hlt/Opp: 0 )
Maurice Ffrench: 0 car 0 yards 0 TDs 0 Fum ( 0 YPC , 0 line YPC , SR%: 0 , Opp Rate: 0 , Hlt/Opp: 0 )
Shocky Jacques-Louis: 0 car 0 yards 0 TDs 0 Fum

In [66]:
## Passing Stats
pass_attempts = pass_plays[
    (pass_plays.play_type == 'Pass Reception')
    | (pass_plays.play_type == 'Passing Touchdown')
    | (pass_plays.play_type == 'Pass Incompletion')
    | (pass_plays.play_type.str.contains('Interception'))
]

pass_completions = pass_attempts[
    (pass_attempts.play_type == 'Pass Reception')
    | (pass_attempts.play_type == 'Passing Touchdown')
]

for team in [away_team, home_team]:
    print(team + ":", 
      len(pass_completions[(pass_completions.offense == team)]),
      "/",len(pass_attempts[(pass_attempts.offense == team)]),
      (sum(pass_completions[(pass_completions.offense == team)].yards_gained)),"yards",
      len(pass_completions[(pass_completions.offense == team) & (pass_completions.play_type == "Passing Touchdown")]),"TDs",
      len(pass_attempts[(pass_attempts.offense == team) & (pass_attempts.play_type.str.contains("Interception"))]),"INTs",
      len(pass_plays[(pass_plays.offense == team) & (pass_plays.play_type.str.contains("Sack"))]),"Sck",
         sum(pass_plays[(pass_plays.offense == team) & (pass_plays.play_type.str.contains("Sack"))].yards_gained),"Sck Yds")

print("")

# look these up manually
away_team_qbs = ["Kenny Pickett"]
home_team_qbs = ["James Graham","Lucas Johnson"]

print(away_team,"Passing Stats")
for qb in away_team_qbs:
    print(qb + ":", 
          len(pass_completions[pass_completions.play_text.str.contains(qb)]),
          "/",len(pass_attempts[pass_attempts.play_text.str.contains(qb)]),
          sum(pass_completions[pass_completions.play_text.str.contains(qb)].yards_gained),"yards",
          len(pass_completions[(pass_completions.play_text.str.contains(qb)) & (pass_completions.play_type == "Passing Touchdown")]),"TDs",
          len(pass_attempts[(pass_attempts.play_text.str.contains(qb)) & (pass_attempts.play_type.str.contains("Interception"))]),"INTs",
         len(pass_plays[(pass_plays.play_text.str.contains(qb)) & (pass_plays.play_type.str.contains("Sack"))]),"Sck",
         sum(pass_plays[(pass_plays.play_text.str.contains(qb)) & (pass_plays.play_type.str.contains("Sack"))].yards_gained),"Sck Yds")
print("")
print(home_team,"Passing Stats")    
for qb in home_team_qbs:
    print(qb + ":", 
          len(pass_completions[pass_completions.play_text.str.contains(qb)]),
          "/",len(pass_attempts[pass_attempts.play_text.str.contains(qb)]),
          sum(pass_completions[pass_completions.play_text.str.contains(qb)].yards_gained),"yards",
          len(pass_completions[(pass_completions.play_text.str.contains(qb)) & (pass_completions.play_type == "Passing Touchdown")]),"TDs",
          len(pass_attempts[(pass_attempts.play_text.str.contains(qb)) & (pass_attempts.play_type.str.contains("Interception"))]),"INTs",
         len(pass_plays[(pass_plays.play_text.str.contains(qb)) & (pass_plays.play_type.str.contains("Sack"))]),"Sck",
         sum(pass_plays[(pass_plays.play_text.str.contains(qb)) & (pass_plays.play_type.str.contains("Sack"))].yards_gained),"Sck Yds")

Alabama: 26 / 39 350 yards 4 TDs 2 INTs 1 Sck -8 Sck Yds
Auburn: 14 / 30 174 yards 1 TDs 0 INTs 0 Sck 0 Sck Yds

Alabama Passing Stats
Kenny Pickett: 0 / 0 0 yards 0 TDs 0 INTs 0 Sck 0 Sck Yds

Auburn Passing Stats
James Graham: 0 / 0 0 yards 0 TDs 0 INTs 0 Sck 0 Sck Yds
Lucas Johnson: 0 / 0 0 yards 0 TDs 0 INTs 0 Sck 0 Sck Yds


In [67]:
# Passing Stats on Std downs
pass_attempts = standard_downs[
    (standard_downs.play_type == 'Pass Reception')
    | (standard_downs.play_type == 'Passing Touchdown')
    | (standard_downs.play_type == 'Pass Incompletion')
    | (standard_downs.play_type.str.contains('Interception'))
#     | (pass_plays.play_type.str.contains('Sack'))
]

pass_completions = standard_downs[
    (standard_downs.play_type == 'Pass Reception')
    | (standard_downs.play_type == 'Passing Touchdown')
]

print("Std Downs")

for team in [away_team, home_team]:
    print(team + ":", 
      len(pass_completions[(pass_completions.offense == team)]),
      "/",len(pass_attempts[(pass_attempts.offense == team)]),
      sum(pass_completions[(pass_completions.offense == team)].yards_gained),"yards",
      len(pass_completions[(pass_completions.offense == team) & (pass_completions.play_type == "Passing Touchdown")]),"TDs",
      len(pass_attempts[(pass_attempts.offense == team) & (pass_attempts.play_type.str.contains("Interception"))]),"INTs",
      len(standard_downs[(standard_downs.offense == team) & (standard_downs.play_type.str.contains("Sack"))]),"Sck",
         sum(standard_downs[(standard_downs.offense == team) & (standard_downs.play_type.str.contains("Sack"))].yards_gained),"Sck Yds")

print("")

print(away_team,"Passing Stats on Std Downs")
for qb in away_team_qbs:
    print(qb + ":", 
          len(pass_completions[pass_completions.play_text.str.contains(qb)]),
          "/",len(pass_attempts[pass_attempts.play_text.str.contains(qb)]),
          sum(pass_completions[pass_completions.play_text.str.contains(qb)].yards_gained),"yards",
          len(pass_completions[(pass_completions.play_text.str.contains(qb)) & (pass_completions.play_type == "Passing Touchdown")]),"TDs",
          len(pass_attempts[(pass_attempts.play_text.str.contains(qb)) & (pass_attempts.play_type.str.contains("Interception"))]),"INTs",
         len(standard_downs[(standard_downs.play_text.str.contains(qb)) & (standard_downs.play_type.str.contains("Sack"))]),"Sck",
          sum(standard_downs[(standard_downs.play_text.str.contains(qb)) & (standard_downs.play_type.str.contains("Sack"))].yards_gained),"Sck Yds")
print("")
print(home_team,"Passing Stats on Std Downs")    
for qb in home_team_qbs:
    print(qb + ":", 
          len(pass_completions[pass_completions.play_text.str.contains(qb)]),
          "/",len(pass_attempts[pass_attempts.play_text.str.contains(qb)]),
          sum(pass_completions[pass_completions.play_text.str.contains(qb)].yards_gained),"yards",
          len(pass_completions[(pass_completions.play_text.str.contains(qb)) & (pass_completions.play_type == "Passing Touchdown")]),"TDs",
          len(pass_attempts[(pass_attempts.play_text.str.contains(qb)) & (pass_attempts.play_type.str.contains("Interception"))]),"INTs",
         len(standard_downs[(standard_downs.play_text.str.contains(qb)) & (standard_downs.play_type.str.contains("Sack"))]),"Sck",
         sum(standard_downs[(standard_downs.play_text.str.contains(qb)) & (standard_downs.play_type.str.contains("Sack"))].yards_gained),"Sck Yds")

Std Downs
Alabama: 16 / 23 242 yards 4 TDs 1 INTs 1 Sck -8 Sck Yds
Auburn: 8 / 14 75 yards 1 TDs 0 INTs 0 Sck 0 Sck Yds

Alabama Passing Stats on Std Downs
Kenny Pickett: 0 / 0 0 yards 0 TDs 0 INTs 0 Sck 0 Sck Yds

Auburn Passing Stats on Std Downs
James Graham: 0 / 0 0 yards 0 TDs 0 INTs 0 Sck 0 Sck Yds
Lucas Johnson: 0 / 0 0 yards 0 TDs 0 INTs 0 Sck 0 Sck Yds


In [68]:
# Passing Stats on Pass. downs
pass_attempts = passing_downs[
    (passing_downs.play_type == 'Pass Reception')
    | (passing_downs.play_type == 'Passing Touchdown')
    | (passing_downs.play_type == 'Pass Incompletion')
    | (passing_downs.play_type.str.contains('Interception'))
#     | (passing_downs.play_type.str.contains('Sack'))
]

pass_completions = passing_downs[
    (passing_downs.play_type == 'Pass Reception')
    | (passing_downs.play_type == 'Passing Touchdown')
]


print("Pass Downs")

for team in [away_team, home_team]:
    print(team + ":", 
      len(pass_completions[(pass_completions.offense == team)]),
      "/",len(pass_attempts[(pass_attempts.offense == team)]),
      sum(pass_completions[(pass_completions.offense == team)].yards_gained),"yards",
      len(pass_completions[(pass_completions.offense == team) & (pass_completions.play_type == "Passing Touchdown")]),"TDs",
      len(pass_attempts[(pass_attempts.offense == team) & (pass_attempts.play_type.str.contains("Interception"))]),"INTs",
      len(passing_downs[(passing_downs.offense == team) & (passing_downs.play_type.str.contains("Sack"))]),"Sck",
         sum(passing_downs[(passing_downs.offense == team) & (passing_downs.play_type.str.contains("Sack"))].yards_gained),"Sck Yds")

print("")

print(away_team,"Passing Stats on Passing Downs")
for qb in away_team_qbs:
    print(qb + ":", 
          len(pass_completions[pass_completions.play_text.str.contains(qb)]),
          "/",len(pass_attempts[pass_attempts.play_text.str.contains(qb)]),
          sum(pass_completions[pass_completions.play_text.str.contains(qb)].yards_gained),"yards",
          len(pass_completions[(pass_completions.play_text.str.contains(qb)) & (pass_completions.play_type == "Passing Touchdown")]),"TDs",
          len(pass_attempts[(pass_attempts.play_text.str.contains(qb)) & (pass_attempts.play_type.str.contains("Interception"))]),"INTs",
          len(passing_downs[(passing_downs.play_text.str.contains(qb)) & (passing_downs.play_type.str.contains("Sack"))]),"Sck",
          sum(passing_downs[(passing_downs.play_text.str.contains(qb)) & (passing_downs.play_type.str.contains("Sack"))].yards_gained),"Sck Yds")
print("")
print(home_team,"Passing Stats on Passing Downs")    
for qb in home_team_qbs:
    print(qb + ":", 
          len(pass_completions[pass_completions.play_text.str.contains(qb)]),
          "/",len(pass_attempts[pass_attempts.play_text.str.contains(qb)]),
          sum(pass_completions[pass_completions.play_text.str.contains(qb)].yards_gained),"yards",
          len(pass_completions[(pass_completions.play_text.str.contains(qb)) & (pass_completions.play_type == "Passing Touchdown")]),"TDs",
          len(pass_attempts[(pass_attempts.play_text.str.contains(qb)) & (pass_attempts.play_type.str.contains("Interception"))]),"INTs",
         len(passing_downs[(passing_downs.play_text.str.contains(qb)) & (passing_downs.play_type.str.contains("Sack"))]),"Sck",
          sum(passing_downs[(passing_downs.play_text.str.contains(qb)) & (passing_downs.play_type.str.contains("Sack"))].yards_gained),"Sck Yds")

Pass Downs
Alabama: 10 / 16 108 yards 0 TDs 1 INTs 0 Sck 0 Sck Yds
Auburn: 6 / 16 99 yards 0 TDs 0 INTs 0 Sck 0 Sck Yds

Alabama Passing Stats on Passing Downs
Kenny Pickett: 0 / 0 0 yards 0 TDs 0 INTs 0 Sck 0 Sck Yds

Auburn Passing Stats on Passing Downs
James Graham: 0 / 0 0 yards 0 TDs 0 INTs 0 Sck 0 Sck Yds
Lucas Johnson: 0 / 0 0 yards 0 TDs 0 INTs 0 Sck 0 Sck Yds


In [69]:
##### Recieving Stats
pass_attempts = base_pbp_data[
    (base_pbp_data.play_type == 'Pass Reception')
    | (base_pbp_data.play_type == 'Passing Touchdown')
    | (base_pbp_data.play_type == 'Pass Incompletion')
    | (base_pbp_data.play_type.str.contains('Interception'))
]

pass_completions = pass_attempts[
    (pass_attempts.play_type == 'Pass Reception')
    | (pass_attempts.play_type == 'Passing Touchdown')
]

print("Receiving Stats")
# look these up manually
away_team_wrs = ["Maurice Ffrench","Taysir Mack","Shocky Jacques-Louis","Nakia Griffin-Stewart","V'Lique Carter","Will Gragg"]
home_team_wrs = ["Adonicas Sanders","Malachi Carter","Ahmarean Brown","Tobias Oliver","Tyler Davis","Nathan Cottrell"]

print(away_team)
for wr in away_team_wrs:
    print(f"{wr}: {len(pass_attempts[pass_attempts.play_text.str.contains(wr)])} tgt, {len(pass_completions[pass_completions.play_text.str.contains(wr)])} catch, {sum(pass_completions[pass_completions.play_text.str.contains(wr)].yards_gained)} yds, SR%: {verify_division(len(pass_attempts[(pass_attempts.play_text.str.contains(wr) & (pass_attempts.play_successful == True))]),len(pass_attempts[pass_attempts.play_text.str.contains(wr)]))}")

print("")

print(home_team)   
for wr in home_team_wrs:
    print(f"{wr}: {len(pass_attempts[pass_attempts.play_text.str.contains(wr)])} tgt, {len(pass_completions[pass_completions.play_text.str.contains(wr)])} catch, {sum(pass_completions[pass_completions.play_text.str.contains(wr)].yards_gained)} yds, SR%: {verify_division(len(pass_attempts[(pass_attempts.play_text.str.contains(wr) & (pass_attempts.play_successful == True))]),len(pass_attempts[pass_attempts.play_text.str.contains(wr)]))}")

Receiving Stats
Alabama
Maurice Ffrench: 0 tgt, 0 catch, 0 yds, SR%: 0
Taysir Mack: 0 tgt, 0 catch, 0 yds, SR%: 0
Shocky Jacques-Louis: 0 tgt, 0 catch, 0 yds, SR%: 0
Nakia Griffin-Stewart: 0 tgt, 0 catch, 0 yds, SR%: 0
V'Lique Carter: 0 tgt, 0 catch, 0 yds, SR%: 0
Will Gragg: 0 tgt, 0 catch, 0 yds, SR%: 0

Auburn
Adonicas Sanders: 0 tgt, 0 catch, 0 yds, SR%: 0
Malachi Carter: 0 tgt, 0 catch, 0 yds, SR%: 0
Ahmarean Brown: 0 tgt, 0 catch, 0 yds, SR%: 0
Tobias Oliver: 0 tgt, 0 catch, 0 yds, SR%: 0
Tyler Davis: 0 tgt, 0 catch, 0 yds, SR%: 0
Nathan Cottrell: 0 tgt, 0 catch, 0 yds, SR%: 0


In [70]:
# conv_down_plays = base_pbp_data[base_pbp_data.down <= 4]
def generate_conv_down_stats(team):
    print(f"Conv Down Stats for {team}")
    team_off_plays = base_pbp_data[base_pbp_data.offense == team]
    team_conv_downs = team_off_plays[team_off_plays.down >= 3]
    succ_conv = team_conv_downs[team_conv_downs.yards_gained >= team_conv_downs.distance]
    team_drives = drives[drives.offense == team]
    print(f"Conv Downs: {len(team_conv_downs)}")
    print(f"Yds/CV: {verify_division(sum(team_conv_downs.distance),len(team_conv_downs))}")
    print(f"Conv Down %: {verify_division(len(team_conv_downs),sum(team_drives.plays))}")
    print(f"Conv/Drive: {verify_division(len(team_conv_downs),len(team_drives))}")
    print(f"Succ Conv: {len(succ_conv)}")
    print(f"Succ Conv %: {verify_division(len(succ_conv),len(team_conv_downs))}")
    print(f"Yds/SC: {verify_division(sum(succ_conv.distance),len(succ_conv))}")
    print(f"SC/Drive: {verify_division(len(succ_conv),len(team_drives))}")
    
generate_conv_down_stats(away_team)
print("\n---\n")
generate_conv_down_stats(home_team)
    

Conv Down Stats for Alabama
Conv Downs: 16
Yds/CV: 7.5
Conv Down %: 0.20253164556962025
Conv/Drive: 1.2307692307692308
Succ Conv: 9
Succ Conv %: 0.5625
Yds/SC: 4.777777777777778
SC/Drive: 0.6923076923076923

---

Conv Down Stats for Auburn
Conv Downs: 13
Yds/CV: 9.0
Conv Down %: 0.18840579710144928
Conv/Drive: 1.0833333333333333
Succ Conv: 5
Succ Conv %: 0.38461538461538464
Yds/SC: 7.6
SC/Drive: 0.4166666666666667


In [71]:
###### Explosiveness rates
# Defined as rate of 15+ yards gains
def generate_exp_rate_stats(team):
    team_plays = base_pbp_data[(base_pbp_data.offense == team)]
    exp_plays = team_plays[(team_plays.play_explosive == True)]
    print(f"Exp Rate Stats for {team}")
    print(f"Total Exp Plays: {len(exp_plays)}")
    print(f"Overall Exp %: {verify_division(len(exp_plays),len(team_plays))}")
    print(f"Pass Exp %: {verify_division(len(exp_plays[(exp_plays.play_type.isin(pass_types))]),len(team_plays[(team_plays.play_type.isin(pass_types))]))}")
    print(f"Pass Exp % (Std Downs): {verify_division(len(standard_downs[(standard_downs.offense == team) & (standard_downs.play_explosive == True) & (standard_downs.play_type.isin(pass_types))]),len(standard_downs[(standard_downs.offense == team) & (standard_downs.play_type.isin(pass_types))]))}")
    print(f"Pass Exp % (Pas Downs): {verify_division(len(passing_downs[(passing_downs.offense == team) & (passing_downs.play_explosive == True) & (passing_downs.play_type.isin(pass_types))]),len(passing_downs[(passing_downs.offense == team) & (passing_downs.play_type.isin(pass_types))]))}")
    print(f"Rush Exp %: {verify_division(len(exp_plays[(exp_plays.play_type.isin(rush_types))]),len(team_plays[(team_plays.play_type.isin(rush_types))]))}")

print("Explosiveness Rates")
generate_exp_rate_stats(away_team)
print("\n---\n")
generate_exp_rate_stats(home_team)

Explosiveness Rates
Exp Rate Stats for Alabama
Total Exp Plays: 14
Overall Exp %: 0.18181818181818182
Pass Exp %: 0.275
Pass Exp % (Std Downs): 0.25
Pass Exp % (Pas Downs): 0.1875
Rush Exp %: 0.08108108108108109

---

Exp Rate Stats for Auburn
Total Exp Plays: 8
Overall Exp %: 0.12307692307692308
Pass Exp %: 0.13333333333333333
Pass Exp % (Std Downs): 0.14285714285714285
Pass Exp % (Pas Downs): 0.125
Rush Exp %: 0.11764705882352941


##### 